**INTRODUCTION/BUSINESS PROBLEM**

In Lagos, Nigeria, Chinese restaurants abound in several locations. However, due to their clientele base over 70% of these restaurants are congregated on the Lagos Island area and some are quite expensive to afford. A new Chinese restaurant franchise wants to make an entry into Lagos(which is the business capital of Nigeria) but would like to understand the terrain before choosing a specific location where they can locate their first branch.

The problems they are faced with right now include;
1. Information of their top 10 competitors presently.
2. Where to locate the restaurant which should be central enough(i.e cinemas, hotels should be close by) and accessiblility to;
  * **young adults in schools,universities**.
  * **relatively affordable to all classes in comparison to the Top 10 competitors**.

**DATASET REQUIRED**

To carry out this excercise, I shall be using numerous dataset sourced from Foursquare, Wikipedia amongst others. I will visualise all the restaurants as well as Chinese restaurants located in Lagos presently using the Foursquare API and cluster them into their various neighborhoods.

I will then explore these restaurants and use reviews and tips available using the Foursquare API to determine the `Top 10 restaurants.`

Afterwards, I will explore the venues around the existing restaurants and determine the best fit using ``Kmeans`` clustering Machine Learning algorithm to generate possiblle locations where the new Chinese restaurant can be situated.

In [5]:
#Let's import some of the libraries I am going to use

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium
import requests
import random
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print("Libraries imported")

Libraries imported


**RESTAURANTS IN LAGOS**

The following codes below will assist me in generating all the restaurants found in Lagos so we can compare with the total number of Chinese restaurants found in the city. I made us of the Foursquare API to locate Lagos and then search for all restaurants within a radius of 10,000m. The data is sent in .json format,normalised and the dataframe using the pandas library is generated.

In [43]:
#Foursquare API Credentials

CLIENT_ID = 'PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ' # my Foursquare ID
CLIENT_SECRET = 'F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW' # my Foursquare Secret
ACCESS_TOKEN = '1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ' # my FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ
CLIENT_SECRET:F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW


In [44]:
#Let's get the lat and long of Lagos, Nigeria using Geopy
address = 'Lagos,Nigeria'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

6.4550575 3.3941795


In [67]:
search_query = 'restaurant'
radius = 10000
print(search_query + ' .... OK!')

restaurant .... OK!


In [69]:
#Let's define the url that will access the API
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUMKFPLVLMDBHJYWINATOBDRDN5FGBEYVJGNRNG2QIFRQDIQ&client_secret=F20KLNJFEZNVQTRNFVAI4Z14IVMT2AMARRESXAT1CHCVVVYW&ll=6.4550575,3.3941795&oauth_token=1XWQ3JFAXHVWDJ3WFUHS0VC0SKXTGEFIZGHXAD5CKPZWAVVZ&v=20180604&query=restaurant&radius=10000&limit=100'

In [70]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607eb77c50b1867ab7eabd59'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '507444a1e4b0740da8b54064',
    'name': 'MacBells Bar & Restaurant',
    'location': {'address': '22 Yaya Abatan Street',
     'lat': 6.4530556,
     'lng': 3.3958333,
     'labeledLatLngs': [{'label': 'display',
       'lat': 6.4530556,
       'lng': 3.3958333}],
     'distance': 288,
     'cc': 'NG',
     'city': 'Ogba',
     'state': 'Lagos',
     'country': 'Nigeria',
     'formattedAddress': ['22 Yaya Abatan Street', 'Ogba', 'Lagos']},
    'categories': [{'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'shortName': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1618917244',
    'hasPerk': False},
   {'id': '4d8f94641716a143da394ef7',
    'name': 'Karma Rest

In [71]:
# Let us assign relevant part of JSON to venues
venues = results['response']['venues']

# Let's tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.tail()

C:\Users\BEDC\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode,venuePage.id,location.neighborhood
45,50fbcdf4e4b0ef4b9ae910ca,Zhou Jie Hot Pot Restaurant,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1618917244,False,NaN,6.427191,3.418003,"[{'label': 'display', 'lat': 6.427190617783614...",4070,NG,Lagos,Lagos,Nigeria,"[Lagos, Lagos]",NaN,NaN,NaN,NaN
46,5854432ece66aa0db6b0aff9,1004 Restaurant & Bar,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1618917244,False,Federal Housing Complex Road,6.438492,3.431951,"[{'label': 'display', 'lat': 6.438492, 'lng': ...",4566,NG,Lagos,Lagos,Nigeria,"[Federal Housing Complex Road (A-block gate), ...",A-block gate,NaN,NaN,NaN
47,53eb6c43498ea98fb87b2524,Coral Blue Seafood Restaurant Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",v-1618917244,False,"1, Adeyemo Alakija Street",6.431819,3.424215,"[{'label': 'display', 'lat': 6.431818578507485...",4210,NG,Lagos,Lagos,Nigeria,"[1, Adeyemo Alakija Street (Victoria Island), ...",Victoria Island,NaN,NaN,NaN
48,4e514b8d7d8bd425e4a9a453,Roots Restaurant,[],v-1618917244,False,82 Adetokunboh St.,6.428380,3.429240,"[{'label': 'display', 'lat': 6.42838, 'lng': 3...",4884,NG,Victoria Island,Lagos,Nigeria,"[82 Adetokunboh St., Victoria Island, Lagos]",NaN,NaN,NaN,NaN
49,51ed39d5498e9269b738ddec,Star Restaurant,"[{'id': '4bf58dd8d48988d1c8941735', 'name': 'A...",v-1618917244,False,NaN,6.422980,3.415400,"[{'label': 'display', 'lat': 6.42298, 'lng': 3...",4273,NG,NaN,NaN,Nigeria,NaN,NaN,NaN,NaN,NaN


In [72]:
dataframe.shape

(50, 19)